## This notebook is a Post-Processing part. Just Follow Our Step!

import module

In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio
import os

from shapely import Polygon
from shapely.geometry import MultiPolygon
from shapely.geometry import box

## Post Processing Function

introduction

1. elipsoid_calculate(),  this func used to calculate polygon's width and height ratio.
2. select_polygons_based_on_condition(), this func used to eliminate polygon which unsuitable for area & width, height ratio condition.
3. extract_mean_ndwi() + mean_thresholding(), this func used to eliminate the lakes which lakes NDWI band's mean value under our threshold.

In [2]:
def elipsoid_calculate(polygon):
    exterior_ring = polygon.exterior
    envelope = exterior_ring.envelope
    envelope_coords = list(envelope.exterior.coords)

    r_x = (envelope_coords[2][0] - envelope_coords[0][0]) / 2
    r_y = (envelope_coords[2][1] - envelope_coords[0][1]) / 2

    if r_x/r_y > 10 or r_y/r_x > 10:
        return False
    else:
        return True

In [3]:
def extract_mean_ndwi(bbox, geotiff_data, geotiff_transform):
    minx, miny, maxx, maxy = bbox
    col_start, row_start = ~geotiff_transform * (minx, maxy)
    col_end, row_end = ~geotiff_transform * (maxx, miny)
    row_start, row_end = int(row_start), int(row_end)
    col_start, col_end = int(col_start), int(col_end)
    clipped_data = geotiff_data[row_start:row_end, col_start:col_end]
    return np.mean(clipped_data)

In [4]:
def select_polygons_based_on_condition(input_path, threshold=100000.0, buf_dist=30):

    threshold = 100000.0   # 0.1 km^2 이상 면적
    buf_dist = 30 # 한 칸의 대각선의 길이가 대략 20m인 점을 고려

    poly_arr = []

    gdf = gpd.read_file(input_path)
    gdf_buffered = gdf.explode(index_parts=True).buffer(buf_dist)
    gdf_buffered = gpd.GeoDataFrame(geometry=gdf_buffered)
    combined_polygons = gdf_buffered.dissolve().explode(index_parts=True).buffer(-buf_dist)

    for poly in combined_polygons:

        if poly.geom_type == 'Polygon':
            polygon = poly
            if elipsoid_calculate(polygon) and polygon.area >= threshold:
                poly_arr.append(polygon)

        elif poly.geom_type == 'MultiPolygon':
            for polygon in poly.geoms:
                if elipsoid_calculate(polygon) and polygon.area >= threshold:
                    poly_arr.append(polygon)

    gdf = gpd.GeoDataFrame(geometry=poly_arr)
    gdf.crs = "EPSG:3857"

    return gdf

In [5]:
def mean_thresholding(Image_path, gdf):
    with rasterio.open(Image_path,'r+') as src:
        geotiff_profile = src.profile
        geotiff_data = src.read(6)  # Assuming 1-band GeoTIFF
        src.crs = gdf.crs

    Image_name = Image_path.split("/")[-1]

    filtered_multipolygons = []
    
    for idx, row in gdf.iterrows():
        bbox = row['geometry'].bounds
        mean_ndwi = extract_mean_ndwi(bbox, geotiff_data, geotiff_profile["transform"])
        #print(mean_ndwi)
        
        if mean_ndwi > 0.3:
            filtered_multipolygons.append(row)

    # 필터링된 MultiPolygon으로 GeoDataFrame 생성
    filtered_gdf = gpd.GeoDataFrame(filtered_multipolygons, crs=gdf.crs, geometry='geometry')
    filtered_gdf['Image'] = Image_name
    # Step 6: 필터링된 MultiPolygon을 저장
    return filtered_gdf

## Read region shape file

This file exist in our github. you can download it as our github repository folder named "region_shape_file"  
After you download the shpaefile, add on region_file pathes. 

In [7]:
region1_file ="C:/Users/scsi/Desktop/Jiwan/SIG/forGIT/SIGspatial/region_shape_file/region_num_1.shp"
region3_file ="C:/Users/scsi/Desktop/Jiwan/SIG/forGIT/SIGspatial/region_shape_file/region_num_3.shp"
region5_file ="C:/Users/scsi/Desktop/Jiwan/SIG/forGIT/SIGspatial/region_shape_file/region_num_5.shp"

region2_file ="C:/Users/scsi/Desktop/Jiwan/SIG/forGIT/SIGspatial/region_shape_file/region_num_2.shp"
region4_file ="C:/Users/scsi/Desktop/Jiwan/SIG/forGIT/SIGspatial/region_shape_file/region_num_4.shp"
region6_file ="C:/Users/scsi/Desktop/Jiwan/SIG/forGIT/SIGspatial/region_shape_file/region_num_6.shp"

region1 = gpd.read_file(region1_file)
region3 = gpd.read_file(region3_file)
region5 = gpd.read_file(region5_file)

region2 = gpd.read_file(region2_file)
region4 = gpd.read_file(region4_file)
region6 = gpd.read_file(region6_file)

add_region(), this function use to add attribute_table column named 'region'

In [8]:
def add_region(gdf, save_path):
    
    intersection1 = gpd.overlay(region1, gdf, how = 'intersection' )
    intersection3 = gpd.overlay(region3, gdf, how = 'intersection' )
    intersection5 = gpd.overlay(region5, gdf, how = 'intersection' )

    intersection2 = gpd.overlay(region2, gdf, how = 'intersection' )
    intersection4 = gpd.overlay(region4, gdf, how = 'intersection' )
    intersection6 = gpd.overlay(region6, gdf, how = 'intersection' )

    intersection1 = intersection1.drop(columns=['fid'])
    intersection3 = intersection3.drop(columns=['fid'])
    intersection5 = intersection5.drop(columns=['fid'])

    intersection2 = intersection2.drop(columns=['fid'])
    intersection4 = intersection4.drop(columns=['fid'])
    intersection6 = intersection6.drop(columns=['fid'])

    merged_intersection = gpd.GeoDataFrame(pd.concat([intersection1, intersection3, intersection5,intersection2, intersection4, intersection6], ignore_index=True))
    merged_intersection.to_file(save_path, driver="GPKG")

## Input the pathdd

 
 1. Image_path, You can download original .tif file in Google drive.  
 2. Input_path, If you learn our model, your result save in that path.  
 3. Save_path, This path is a save root, After our post-processing in each .gpkg file.dd

In [14]:
Image_path = "D:/SIG/9.final_data/Greenland_sentinel2_19-06-03_region_5.tif" # Original Image path, careful for date-region.
input_path = "C:/Users/scsi/Desktop/result/0603_region5.gpkg" #your model result gpkg path, also careful for date-region.
save_path = "C:/Users/scsi/Desktop/result/final_gpkg/0603_region5_final.gpkg" # each gpkg saved path.

If all your path setting is correct, this code will generate each .gpkf file in your 'save_path' folder.  
Unfortunately, you can edit the all path iteratively, in each implementation.  
After you implemented the under cell, please edit your path!

In [15]:
gdf = select_polygons_based_on_condition(input_path)
gdf = mean_thresholding(Image_path , gdf)
add_region(gdf, save_path)

Finally, Your 'save_path' are filled with each .gpkg files.  
Under cell is a merged all your .gpkg file.  
Input the final path when you saved your merged .gpkg file in 'output_gpkg'.  
Also, Input your folder path which include each .gpkg files, ( = save_path folder)

In [17]:
output_gpkg = "C:/Users/scsi/Desktop/result/final_merged/lake_final.gpkg"
folder_path = "C:/Users/scsi/Desktop/result/final_gpkg/"

all_gpkg_files = []

for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith(".gpkg"):
            gpkg_path = os.path.join(root, file)
            all_gpkg_files.append(gpkg_path)

all_gdfs = []

for gpkg_file in all_gpkg_files:
    gdf = gpd.read_file(gpkg_file)
    all_gdfs.append(gdf)

combined_gdf = gpd.GeoDataFrame(pd.concat(all_gdfs, ignore_index=True))

combined_gdf.to_file(output_gpkg, driver='GPKG')